# Import Dependency

In [2]:
import home_default_package as hd

# Data Acquisition

In [3]:
path = r"C:\Users\nene0\Documents\Datasets\home-credit-default-risk\application_train.csv"
bureau_path = r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_bureau.csv"
credit_path = r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_credit.csv"
instal_path = r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_installment.csv"
cash_path = r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\DataScienceMod2_LFZ\LoanDefault\cleaned_POS.csv"

# Pipeline

In [4]:
train = hd.read_file(path)
bureau = hd.read_file(bureau_path)
credit = hd.read_file(credit_path)
instal = hd.read_file(instal_path)
cash = hd.read_file(cash_path)

train = hd.balance_class(train)
train = hd.remove_outliers(train)
train = hd.days_features(train)


ext2_pred_ft = ['REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','REGION_POPULATION_RELATIVE']
ext2 = ['EXT_SOURCE_2']
ext2_pred = hd.ext_source_knn(train, ext2_pred_ft, ext2)
train = hd.fill_missing_external(train, 'EXT_SOURCE_2', ext2_pred)

ext1_pred_ft = ['AGE', 'EXT_SOURCE_2']
ext1 = ['EXT_SOURCE_1']
ext1_pred = hd.ext_source_knn(train, ext1_pred_ft, ext1)
train = hd.fill_missing_external(train, 'EXT_SOURCE_1', ext1_pred)

ext3_pred_ft = ['AGE', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'DAYS_ID_PUBLISH']
ext3 = ['EXT_SOURCE_3']
ext3_pred = hd.ext_source_knn(train, ext3_pred_ft, ext3)
train = hd.fill_missing_external(train, 'EXT_SOURCE_3', ext3_pred)

train = hd.data_imputation(train)
train = hd.drop_features(train)

train = hd.merge_datasets(train, bureau, credit, instal, cash)
train = hd.feature_engineering(train)

scaled_train = hd.scale_data(train)
encoded_train = hd.encode_data(train)
final_train = hd.generate_final_data(scaled_train, encoded_train)

x_train, x_test, y_train, y_test = hd.split_data(final_train, train)

xgb = hd.XGBClassifier(random_state=0, eta=0.2)
xgb_pred, xgb_pred_proba = hd.train_pred_model(x_train, y_train, x_test, xgb)

ann_model = hd.train_ann(x_train, y_train)
ann_pred = hd.predict_ann(x_test, ann_model)

Epoch 1/100
32/32 [==============================] - 1s 9ms/step - loss: 0.6906 - auc: 0.5484 - val_loss: 0.6888 - val_auc: 0.5643
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6865 - auc: 0.5785 - val_loss: 0.6847 - val_auc: 0.5849
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6818 - auc: 0.5977 - val_loss: 0.6799 - val_auc: 0.6029
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6761 - auc: 0.6178 - val_loss: 0.6737 - val_auc: 0.6223
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6689 - auc: 0.6377 - val_loss: 0.6657 - val_auc: 0.6426
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6598 - auc: 0.6592 - val_loss: 0.6574 - val_auc: 0.6604
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6503 - auc: 0.6758 - val_loss: 0.6490 - val_auc: 0.6744
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6408 - a

In [5]:
xgb_pred

array([1, 0, 1, ..., 0, 0, 1])

In [6]:
hd.print_scores(y_test, xgb_pred)

,Precision,Recall,ROC_AUC
0,0.6962,0.691702,0.69566


In [7]:
ann_pred

array([[0.74011344],
       [0.24031477],
       [0.44202417],
       ...,
       [0.13756593],
       [0.43956795],
       [0.67043835]], dtype=float32)